In [7]:
import xml.dom.minidom

In [109]:
# Can use BeautifulSoup to parse XML
# from bs4 import BeautifulSoup

# Or use Elementtree

def importXML_viaFileReader(fileName):
    with open(fileName, 'r') as f:
        data = f.read()
    return data

def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)


def importXML(fileName):
    return(xml.dom.minidom.parse(fileName))



bpmn = importXML('../bpmn/flight-booking_idea1.bpmn')

In [110]:
print(bpmn.firstChild.tagName)

bpmn:definitions


# Coding

In [111]:
participant = bpmn.getElementsByTagName("bpmn:participant")
for i in participant:
    name = i.getAttribute("name")

serviceTask = bpmn.getElementsByTagName("bpmn:serviceTask")
ms = []
for i in serviceTask :
    ms.append(i.getAttribute("name"))
ms_iterator = [s.replace("\n", " ") for s in ms]
ms = list(ms_iterator)

# Execution Formats
ms_name = []
for i in ms:
    if i.find("[") != -1:
        s = i.find("[")
        t = i.find("]")
        i = i.replace(i[s:t+1], "")
    ms_name.append(i)
ms_name = list(dict.fromkeys(ms_name))

execution_formats = [[] for x in range(len(ms_name))]
for i in ms:
    if i.find("[") != -1:
        s = i.find("[")
        t = i.find("]")
        temp_indexFinder = i.replace(i[s:t+1], "")
        ms_iter = ms_name.index(temp_indexFinder)
        print(ms_iter)
        execution_formats[ms_iter].append(i[s+1:t])
    else:
        temp_indexFinder = i
        ms_iter = ms_name.index(temp_indexFinder)
        execution_formats[ms_iter].append("")
print(ms_name)
print(execution_formats)

0
0
0
2
2
3
3
['Flight Search ', 'Weather Information', 'Flight Booking ', 'Rental Car Booking ', 'Payment']
[['Normal', 'Low Power', 'High Performance'], [''], ['Normal', 'Low power'], ['Normal', 'High Performance'], ['']]


In [112]:
import json

jsObj = {
    "name": name,
    "components": {

    }
}
for i in ms_name:
    jsObj["components"][i] = {
    }
    for j in execution_formats[ms_name.index(i)]:
        jsObj["components"][i][j] = {
                "aws-ec2-type": "",
                "energy-demand": 0.0,
                "user-scaling": 0,
                "q": 0.0
        }


json_object = json.dumps(jsObj, indent = 4)


# Write to json
with open(name + ".json", "w") as outfile:
    outfile.write(json_object)


## Execution Format Data

In [129]:
exclusiveGateway = bpmn.getElementsByTagName("bpmn:exclusiveGateway")
xor_incoming = {}
for i in exclusiveGateway:
    xor_incoming[i.getAttribute("id")] = []
    incomingE = i.getElementsByTagName("bpmn:incoming")
    for j in incomingE:
        xor_incoming[i.getAttribute("id")].append(j.firstChild.nodeValue)
    #print(len(i.getElementsByTagName("bpmn:outgoing")))
xor_outgoing = {}
for i in exclusiveGateway:
    xor_outgoing[i.getAttribute("id")] = []
    outgoingE = i.getElementsByTagName("bpmn:outgoing")
    for j in outgoingE:
        xor_outgoing[i.getAttribute("id")].append(j.firstChild.nodeValue)
    #print(len(i.getElementsByTagName("bpmn:outgoing")))
print(xor_outgoing)

{'Gateway_1m30h1u': ['Flow_00n5nre', 'Flow_0d32kes', 'Flow_1yo3en2'], 'Gateway_0oqidx5': ['Flow_0oo8rar', 'Flow_1vzbicw'], 'Gateway_09s5c9c': ['Flow_00c5ec3', 'Flow_11ukj51'], 'Gateway_0ijge64': ['Flow_1wftvp9'], 'Gateway_1m18p5p': ['Flow_1145dvq'], 'Gateway_0wzzojv': ['Flow_0qh8ldb', 'Flow_010zav3'], 'Gateway_09y8nf2': ['Flow_0kh2ita', 'Flow_07i3ize'], 'Gateway_1evznl6': ['Flow_0z8hi84'], 'Gateway_04u48v1': ['Flow_0bpg9v3']}


In [167]:
# Whenever an xor-Gate iss used to skip a service, that service is optional
for i in serviceTask :
    incomingE = i.getElementsByTagName("bpmn:incoming")
    outgoingE = i.getElementsByTagName("bpmn:outgoing")
    for j in incomingE:
        sMS = j.firstChild.nodeValue
        for x in xor_outgoing.items():
            if sMS in x[1]:
                previousXOR = x[0]
                for tOut in xor_outgoing[previousXOR]:
                    print("Out")
                # TODO: Check if the outgoing edge is the one that skips the service
                ## i.e. it has an direct edge to another Gateway
    for j in outgoingE:
        tMS = j.firstChild.nodeValue
    #previousXOR = xor_outgoing.keys()[xor_outgoing.values().index(sMS)]



Flow_00n5nre
Flow_0d32kes
Flow_1yo3en2
Flow_00n5nre
Flow_0d32kes
Flow_1yo3en2
Flow_00n5nre
Flow_0d32kes
Flow_1yo3en2
Flow_0oo8rar
Flow_1vzbicw
Flow_00c5ec3
Flow_11ukj51
Flow_00c5ec3
Flow_11ukj51
Flow_0kh2ita
Flow_07i3ize
Flow_0kh2ita
Flow_07i3ize
Flow_0bpg9v3
